In [1]:
from classifier_lewidi import *
from params_lewidi import *
from DataManager import *

import os
import argparse
from datetime import datetime
import pytz
import re


In [2]:


filepath1 = '../data/ConvAbuse_dataset/'
filepath2 = '../data/ArMIS_dataset/'
filepath3 = '../data/HS-Brexit_dataset/'
filepath4 = '../data/MD-Agreement_dataset/'
data = DataManager(filepath1, filepath2, filepath3, filepath4)




In [ ]:
print(data)

In [ ]:
def_params = params_lewidi()
for key in data:
    annotators = list(sorted(set(itertools.chain.from_iterable(data[key]['annotators'].str.findall("\w+")))))
    print("-------------------------------------------------------------------------------------------------------") 
    print("-----------------------------WE ARE IN THE", key, "DATASET---------------------------------------------")
    
    model = ToxicityClassifier(data, annotators=annotators, params=def_params)

    score, results = model.CV()

    score["params"] = ", ".join(key + ": " + str(val) for key, val in def_params.__dict__.items())
    score["task"] = model.params.task

    pacific = pytz.timezone('US/Pacific')
    sa_time = datetime.now(pacific)
    name_time = sa_time.strftime('%m%d%y-%H:%M')
    score["time"] = name_time
    print(score)

    score_dir = os.path.join(def_params.source_dir, "results", "GHC", "classification.csv")
    result_dir = os.path.join(def_params.source_dir, "results", "GHC", name_time + "_" + def_params.task + ".csv")
    if os.path.exists(score_dir):
        pd.DataFrame.from_records([score]).to_csv(score_dir, header=False,  index=False, mode="a")
    else:
        pd.DataFrame.from_records([score]).to_csv(score_dir, index=False)

        pd.DataFrame.from_dict(results).to_csv(result_dir, index=False)
    
    print("---------------------------------------", key, "DATASET END--------------------------------------------")
    print("-------------------------------------------------------------------------------------------------------")